In [7]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARMA
from statsmodels.stats.diagnostic import acorr_ljungbox

In [8]:
data = pd.read_csv("Data/VAR_data.csv")

In [9]:
data.head()

,ExchangeRateLog,ExchangeRateLogDifference,GDP,M2,Inflation,ShortInterestRate
0,0.257970,-0.018525,-0.097363,0.123950,0.7691,0.6428
1,0.238781,-0.019189,-0.097363,0.109485,0.7326,0.7144
2,0.195156,-0.043625,-0.097363,0.103728,0.7459,0.8590
3,0.185732,-0.009423,-0.097363,0.100158,0.8640,1.1006
4,0.206608,0.020875,-0.097363,0.110161,0.9445,1.3075


In [10]:
ExchangeRateLog = data['ExchangeRateLog']

In [11]:
ExchangeRateLog_d_1 = data['ExchangeRateLogDifference']

In [13]:
delta_GDP = np.array(data['GDP'])
delta_m2 = np.array(data['M2'])
delta_pie = np.array(data['Inflation'])
delta_r = np.array(data['ShortInterestRate'])

In [14]:
exog_var = np.array([delta_GDP,delta_m2,delta_pie,delta_r])
exog_var = np.transpose(exog_var)

In [15]:
exog_var.shape

(144, 4)

In [40]:
AicScore = pd.DataFrame(index=list(range(5)),columns=list(range(5)))

In [41]:
for p in range(5):
    for q in range(5):
        try:
            model = ARMA(ExchangeRateLog_d_1,order=(p,q),exog=exog_var)
            result = model.fit()
            AicScore.iloc[p,q] = result.aic
        except ValueError:
            AicScore.iloc[p,q] = 1

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retval

In [42]:
AicScore

,0,1,2,3,4
0,-668.736,-675.125,-673.187,-671.255,-672.942
1,-674.142,-673.177,1,1,1
2,-673.042,-678.864,-671.902,1,1
3,-671.746,-669.893,-667.853,-675.891,1
4,-670.487,-674.492,-672.485,-670.47,-667.922


In [19]:
ExchangeRateLog = np.array(ExchangeRateLog)

In [23]:
ExchangeRateLog_d_1 = np.array(ExchangeRateLog_d_1)

In [21]:
exog_var = np.array([delta_GDP,delta_m2,delta_pie,delta_r])
exog_var = np.transpose(exog_var)

In [24]:
#Test ARMA(1,1,1)
model = ARMA(ExchangeRateLog_d_1,order=(1,1),exog=exog_var)
result = model.fit()

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [26]:
result.pvalues

array([ 0.93180181,  0.53782144,  0.81214816,  0.33762353,  0.042965  ,
        0.8158573 ,  0.19608544])

In [27]:
res = result.resid

In [29]:
acorr_ljungbox(res,lags=20)[-1]

array([ 0.99813488,  0.98819935,  0.99664685,  0.74412241,  0.65700297,
        0.69336313,  0.24017604,  0.31674476,  0.28554392,  0.362309  ,
        0.27429858,  0.31337622,  0.23490221,  0.27803026,  0.34238037,
        0.14337392,  0.14571808,  0.13406819,  0.07905405,  0.08239788])

In [30]:
def mean_error(pred,real):
    return np.mean([pred[i]-real[i] for i in range(len(pred))])
def mean_absolute_error(pred,real):
    return np.mean([np.abs(pred[i]-real[i]) for i in range(len(pred))])
def root_mean_square_error(pred,real):
    return np.sqrt(np.mean([pow(pred[i]-real[i],2) for i in range(len(pred))]))

In [31]:
PredictionOutputMAE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])
PredictionOutputRMSE = pd.DataFrame(index=['monthly','3_month','6_month','12_month'])

In [32]:
totalData = 144
startPrediction = 100

In [33]:
tsValuesLog = list(ExchangeRateLog)

In [36]:
tsValuesLog_d_1 = ExchangeRateLog_d_1

In [34]:
len(ExchangeRateLog)

144

In [35]:
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        pred.append(tsValuesLog[startPrediction-1+i])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['RandomWalk'] = mae_
PredictionOutputRMSE['RandomWalk'] = rmse_

In [37]:
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        history = tsValuesLog_d_1[i:i+startPrediction]
        exog_history = exog_var[i:i+startPrediction]
        model = ARMA(endog=history,order=(1,1),exog=exog_history)
        result = model.fit()
        prediction = result.predict(start = startPrediction, end = startPrediction+l-1,exog=exog_var[startPrediction:startPrediction+l])
        pred.append(sum(prediction)+tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['ARMA_1_1+Exo_4'] = mae_
PredictionOutputRMSE['ARMA_1_1+Exo_4'] = rmse_

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

In [33]:
exog_var = np.array([delta_GDP,delta_pie])
exog_var = np.transpose(exog_var)

In [49]:
mae_ = []
rmse_ = []
for l in [1,3,6,12]:
    pred = [0]*(startPrediction+l-1)
    for i in range(totalData-startPrediction-l+1):
        
        try:
            history = tsValuesLog[i:i+startPrediction]
            exog_history = exog_var[i:i+startPrediction]
            model = ARMA(endog=history,order=(2,1),exog=exog_history)
            result = model.fit()
            prediction = result.predict(start = startPrediction, end = startPrediction+l-1,exog=exog_var[startPrediction:startPrediction+l])
            pred.append(sum(prediction)+tsValuesLog[i+startPrediction-1])
        except np.linalg.LinAlgError:
            pred.append(tsValuesLog[i+startPrediction-1])
    mae_.append(mean_absolute_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
    rmse_.append(root_mean_square_error(pred[startPrediction+l-1:],tsValuesLog[startPrediction+l-1:]))
PredictionOutputMAE['ARMA_2_1+Exo_4'] = mae_
PredictionOutputRMSE['ARMA_2_1+Exo_4'] = rmse_

C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Hallo\Anaconda3\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check

In [50]:
PredictionOutputMAE

,RandomWalk,ARMA_1_1+Exo_4,ARMA_2_1+Exo_4
monthly,0.014526,0.015745,0.242317
3_month,0.031639,0.033484,0.761628
6_month,0.051876,0.055610,1.562381
12_month,0.085842,0.094035,3.151553


In [51]:
PredictionOutputRMSE

,RandomWalk,ARMA_1_1+Exo_4,ARMA_2_1+Exo_4
monthly,0.019400,0.020178,0.249965
3_month,0.042021,0.043078,0.778293
6_month,0.069462,0.072504,1.592741
12_month,0.116094,0.123886,3.214722


In [36]:
PredictionOutputMAE.to_csv("Result/Fit_Macro_Exogenous_mae.csv")
PredictionOutputRMSE.to_csv("Result/Fit_Macro_Exogenous_rmse.csv")